In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import math, nltk, warnings
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet
import warnings; warnings.simplefilter('ignore')

In [ ]:
import matplotlib as fig

In [ ]:
pip install fuzzywuzzy


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math, nltk, warnings
from nltk.corpus import wordnet
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors
from wordcloud import WordCloud, STOPWORDS
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings('ignore')
PS = nltk.stem.PorterStemmer()



# Recommendation Model


In [5]:
df = pd.read_csv("/content/drive/MyDrive/clean_data.csv")

In [ ]:
gs_filter = lambda x,y,sum: math.exp(-(x-y)**2/(2*sum**2))

In [ ]:
def variables_at_entry(df, identification):
  columns = []
  if pd.notnull(df['director_name'].iloc[identification]):
    for i in df['director_name'].iloc[identification].split('|'):
      columns.append(i)


  
    if pd.notnull(df['actor_1_name'].iloc[identification]):
            for s in df['actor_1_name'].iloc[identification].split('|'):
                columns.append(s)
    if pd.notnull(df['actor_2_name'].iloc[identification]):
            for s in df['actor_2_name'].iloc[identification].split('|'):
                columns.append(s)         
    if pd.notnull(df['actor_3_name'].iloc[identification]):
            for s in df['actor_3_name'].iloc[identification].split('|'):
                columns.append(s)
    

  if pd.notnull(df['keywords'].iloc[identification]):
    for i in df['keywords'].iloc[identification].split('|'):
      columns.append(i)

  return columns

In [ ]:
def variables_adding(df, reference):
  for i in reference: df[i] = pd.Series([0 for _ in range(len(df))])
  cols = ['genres','actor_1_name', 'actor_2_name', 'actor_3_name', 'director_name', 'keywords']

  for group in cols:
    for position, row in df.iterrows():
      if pd.isnull(row[group]):
        continue
      for i in row[group].split('|'):
        if i in reference:
          df.at[position, i] = 1

  return df

In [ ]:
def suggestion(df, identification):
  df_new = df.copy(deep = True)
  genres = set()
  for i in df['genres'].str.split('|').values:
    genres = genres.union(set(i))

  #Similarity check
  identifiers = variables_at_entry(df_new, identification)
  identifiers += list(genres)
  new_dataframe = variables_adding(df_new, identifiers)

  #Closest neighbours
  y = new_dataframe.values
  neighbours = NearestNeighbors(n_neighbors = 31, algorithm = 'auto', metric = 'euclidean').fit(y)

  dis, pos = neighbours.kneighbors(y)
  ytest = new_dataframe.iloc[identification].values
  ytest = ytest.reshape(1, -1)

  dis, pos = neighbours.kneighbors(ytest)

  return pos[0][:]

In [ ]:

def extraction(df, movies_list):
  movies = ['_' for _ in range(31)]
  j = 0
  maximum_users = -1
  for pos in movies_list:
    movies[j] = list(df.iloc[pos][['movie_title', 'title_year', 'imdb_score', 'num_user_for_reviews', 'num_voted_users']])
    movies[j].append(pos)
    if maximum_users > movies[j][4] :
      maximum_users = maximum_users
    else :
      maximum_users = movies[j][4]    
    j+=1

  headline = movies[0][0]
  ref = movies[0][1]
  movies.sort(key = lambda a:selection(headline, maximum_users, ref, a[0], a[1], a[2], a[4]), reverse = True)

  return movies


In [ ]:
def second_part(title_1, title_2):
  if fuzz.ratio(title_1, title_2) > 50 or fuzz.token_set_ratio(title_1, title_2) > 50:
    return True
  else:
    return False


In [ ]:
def selection(main_title, maximum_users, ref, title, annee, imdb, votes):
  if pd.notnull(ref):
    factor_1 = gs_filter(ref, annee, 20)
  else:
    facotr_1 = 1

  sum = maximum_users*1.0

  if pd.notnull(votes):
    factor_2 = gs_filter(votes, maximum_users, sum)
  else:
    factor_2 = 0

  if second_part(main_title, title):
    record = 0
  else:
    record = imdb**2 * factor_1 * factor_2

  return record

In [ ]:
def adding_to_selection(selection, movie_parameters):
  movies = selection[:]
  counting = len(movies)
  
  for j in range(31):
    present_in_list = False
    for i in selection:
      if i[0] == movie_parameters[i][0]:
        present_in_list = True
      if second_part(movie_parameters[j][0], i[0]):
        present_in_list = True
    if present_in_list == True:
      continue
    counting += 1
    if counting <= 5:
      film_list.append(movie_parameters[j])
    
  return movies

In [ ]:
def remove_second_parts(selecting_films):
  removed = []
  for s, movie_1 in enumerate(selecting_films):
    for t, movie_2 in enumerate(selecting_films):
      if t <= s:
        continue
      if second_part(movie_1[0], movie_2[0]):
        film = movie_2[0] if movie_1[1] < movie_2[1] else movie_1[0]
        removed.append(film)

  movies = [movie for movie in selecting_films if movie[0] not in removed]

  return movies

In [ ]:
def likeness(df, identification, sequels_del = True, ver = False):
  if ver:
    print(90*'_' + '\n' + "Query: films similar to id = {} -> '{}'".format(identification, df.iloc[identification]['title']))
  
  movies = suggestion(df, identification)

  movie_parameters = extraction(df, movies) #List of films

  select_movie = []
  select_movie = adding_to_selection(select_movie, movie_parameters)

  if sequels_del:
    select_movie = remove_second_parts(select_movie)

  select_movie = adding_to_selection(select_movie, movie_parameters)

  title_selection = []
  for p, q in enumerate(select_movie):
    select_movie.append(q[0].replace(u'\xa0', u''), q[5])
    if ver:
      print("nº{:<2}     -> {:<30}".format(p+1, q[0]))

  return title_selection

In [ ]:
choose = dict()
for s in range(0, 10, 3):
    choose[s] = likeness(df, s, sequels_del = True, ver = True)

QUERY: films similar to id=0 -> 'Avatar'
nº1      -> Avatar                      
nº2      -> Star Trek Into Darkness       
nº3      -> Jupiter Ascending             
nº4      -> The Fifth Element             
nº5      -> Pirates of the Caribbean: On Stranger Tides
__________________________________________________________________________________________
QUERY: films similar to id=3 -> 'The Dark Knight Rises'
nº1      -> The Dark Knight               
nº2      -> Batman & Robin                
nº3      -> Kick-Ass                      
nº4      -> Hitman                        
nº5      -> Running Scared                
__________________________________________________________________________________________
QUERY: films similar to id=6 -> 'Tangled'
nº1      -> Tangled                       
nº2      -> The Princess and the Frog     
nº3      -> Ponyo                         
nº4      -> Toy Story 3                   
nº5      -> Pinocchio                     
_______________________